In [25]:
# Useful Resources
# https://www.pyimagesearch.com/2016/02/08/opencv-shape-detection/
# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contour_features/py_contour_features.html
# https://www.learnopencv.com/blob-detection-using-opencv-python-c/
# https://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html
# https://stackoverflow.com/questions/55614910/general-object-counting-python-opencv
# https://www.pyimagesearch.com/2014/08/04/opencv-python-color-detection/
# https://www.pyimagesearch.com/2017/01/02/rotate-images-correctly-with-opencv-and-python/
# https://github.com/Ajay008/OpenCV/blob/master/Color%20Tracker/colorTracker.py

#https://jponttuset.cat/solving-sudokus-like-a-pro-1/
#https://stackoverflow.com/questions/51034209/detect-the-corners-of-a-grid-using-opencv
#https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html
#hough transform opencv
#https://www.learnopencv.com/hough-transform-with-opencv-c-python/
import numpy as np
import cv2

# test image sequence
test_seq = ["/Users/lostll/Desktop/180FinalProject/3bit.jpg","/Users/lostll/Desktop/180FinalProject/1bit.jpg","/Users/lostll/Desktop/180FinalProject/2bit.jpg","/Users/lostll/Desktop/180FinalProject/4bit.jpg"]

# total device number
DEV_NUM = 16

# default size
DEFAULT_SIZE = 600

# Some HSV color bound
HSV_bound = [([0, 0, 200], [100, 200, 255]),
             ([100, 200, 50], [160, 255, 200])] # red + green

G_bound = [([80, 180, 50], [160, 255, 200])]
R_bound = [([0, 0, 200], [100, 200, 255])]

NE = [1, 1]
NW = [-1, 1]
SE = [1, -1]
SW = [-1, -1]

In [26]:
# Take an image and filter according to Boundry.
def color_filter(frame, bound):
    mask = cv2.inRange(frame, np.array([0, 0, 0]), np.array([0, 0, 0]))
    for (lower, upper) in bound:
        lower = np.array(lower)
        upper = np.array(upper)
        # create NumPy arrays from the boundaries
        mask += cv2.inRange(frame, lower, upper)
    bright = cv2.add(frame,np.array([100.0]))
    output = cv2.bitwise_and(bright, bright, mask = mask)
    return output

In [27]:
# Testcode for func color_filter
test_image = cv2.imread("/Users/lostll/Desktop/180FinalProject/init.jpg")
#test_image = cv2.GaussianBlur(test_image, (9, 9), 0)
ftd_image = color_filter(test_image, HSV_bound)

# testing
#image_blur = cv2.GaussianBlur(ftd_image, (5, 5), 0)
#image_gray = cv2.cvtColor(image_blur, cv2.COLOR_BGR2GRAY)
#image_edged = cv2.Canny(image_gray, 100, 200)
#image_edged = cv2.dilate(image_edged, None, iterations=1)
#image_edged = cv2.erode(image_edged, None, iterations=1)
#image_edged = cv2.GaussianBlur(image_gray, (13, 13), 0)
#image_edged = cv2.bitwise_not(image_edged)

#lines = cv2.HoughLines(image_edged,1,np.pi/180,200)
#print(lines)

#for rho,theta in lines[0]:
#    a = np.cos(theta)
#    b = np.sin(theta)
#    x0 = a*rho
#    y0 = b*rho
#    x1 = int(x0 + 1000*(-b))
#    y1 = int(y0 + 1000*(a))
#    x2 = int(x0 - 1000*(-b))
#    y2 = int(y0 - 1000*(a))

#    cv2.line(image_edged,(x1,y1),(x2,y2),(0,0,255),2)

cv2.imwrite('/Users/lostll/Desktop/180FinalProject/180test/filtertest.jpg', ftd_image) 

True

In [28]:
def roi_center(img, size = DEFAULT_SIZE):
    image_blur = cv2.GaussianBlur(img, (3, 3), 0)
    image_gray = cv2.cvtColor(image_blur, cv2.COLOR_BGR2GRAY)
    image_edged = cv2.Canny(image_gray, 100, 200)
    image_edged = cv2.dilate(image_edged, None, iterations=1)
    image_edged = cv2.erode(image_edged, None, iterations=1)
    image_edged = cv2.GaussianBlur(image_gray, (21, 21), 0)
    image_edged = cv2.bitwise_not(image_edged)
    params = cv2.SimpleBlobDetector_Params()
    # Change thresholds
    params.minThreshold = 0;
    params.maxThreshold = 255;
    # Filter by Area.
    params.filterByArea = True
    params.minArea = size
    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0
    # Filter by Convexity
    params.filterByConvexity = True
    params.minConvexity = 0.87
    # Filter by Inertia
    params.filterByInertia = True
    params.minInertiaRatio = 0.01
    # Create a detector with the parameters
    ver = (cv2.__version__).split('.')
    if int(ver[0]) < 3 :
        detector = cv2.SimpleBlobDetector(params)
    else:
        detector = cv2.SimpleBlobDetector_create(params)
    # Test Code
    cv2.imwrite('/Users/lostll/Desktop/180FinalProject/180test/now1.jpg', image_edged) 
    # gives a set of coordinates of device centers
    key_points = detector.detect(image_edged)
    return key_points

In [29]:
# Test code for func roi_center
key_points = roi_center(ftd_image)
count = 0
for i in key_points:
    count = count + 1
print(len(key_points))

im_with_keypoints = cv2.drawKeypoints(ftd_image, key_points, np.array([]), (0, 0, 255),
                                      cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imwrite('/Users/lostll/Desktop/180FinalProject/180test/now.jpg', im_with_keypoints) 

16


True

In [30]:
def init(dev_num = DEV_NUM, size = 600):

    # change to call funcs later
    init_img = cv2.imread("/Users/lostll/Desktop/180FinalProject/init.jpg")
    ftd_img = color_filter(init_img, HSV_bound)
    key_points = roi_center(ftd_img, size)
    
    det = False
    if len(key_points) == dev_num:
        det = True

    return key_points, det

In [31]:
[roi, det] = init()
if det == False:
    print("Some Device Missing")
    roi = []
print(roi)

[<KeyPoint 0x11c45e510>, <KeyPoint 0x11c45e450>, <KeyPoint 0x11c45e540>, <KeyPoint 0x11c45e5a0>, <KeyPoint 0x11c45e570>, <KeyPoint 0x11c45e180>, <KeyPoint 0x11c45e8a0>, <KeyPoint 0x11c45ecc0>, <KeyPoint 0x11c45e1b0>, <KeyPoint 0x11c45e960>, <KeyPoint 0x11c45e810>, <KeyPoint 0x11c45e6f0>, <KeyPoint 0x11c45eae0>, <KeyPoint 0x11c45eb10>, <KeyPoint 0x11c45eb40>, <KeyPoint 0x11c45e7e0>]


In [32]:
def find_nearest(cor, cor_list):
    nearest = cor
    len = 99999999
    for i in cor_list:
        new_len = abs(cor.pt[0] - i.pt[0]) * abs(cor.pt[0] - i.pt[0]) + abs(cor.pt[1] - i.pt[1]) * abs(cor.pt[1] - i.pt[1])
        if new_len < len:
            nearest = i
            len = new_len
    return nearest

In [33]:
def ori_grid(img, dev_center, sgn_bound, default_ori):
    
    ori = {}    
    ori_ftd_image = color_filter(img, sgn_bound)
    ori_key_points = roi_center(ori_ftd_image, 150)
    
    im_with_keypoints = cv2.drawKeypoints(ori_ftd_image, ori_key_points, np.array([]), (0, 0, 255),
                                      cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imwrite('/Users/lostll/Desktop/180FinalProject/180test/now.jpg', im_with_keypoints) 
    
    for i in ori_key_points:
        dir_center = find_nearest(i, dev_center)
        if (dir_center.pt[0] - i.pt[0]) * default_ori[0] + (dir_center.pt[1] - i.pt[1]) * default_ori[1] > 0:
            ori[dir_center] = 1
        else:
            ori[dir_center] = 0

    return ori

In [34]:
print(ori_grid(test_image, roi, G_bound, NW))

{<KeyPoint 0x11c45eae0>: 1, <KeyPoint 0x11c45e6f0>: 1, <KeyPoint 0x11c45e7e0>: 1, <KeyPoint 0x11c45e450>: 1, <KeyPoint 0x11c45e8a0>: 1, <KeyPoint 0x11c45e510>: 1, <KeyPoint 0x11c45ecc0>: 1, <KeyPoint 0x11c45e960>: 1, <KeyPoint 0x11c45e5a0>: 1, <KeyPoint 0x11c45e1b0>: 1, <KeyPoint 0x11c45e540>: 1, <KeyPoint 0x11c45e570>: 1, <KeyPoint 0x11c45e810>: 1, <KeyPoint 0x11c45e180>: 1, <KeyPoint 0x11c45eb10>: 1, <KeyPoint 0x11c45eb40>: 1}


In [35]:
import math

def localization(roi_center_l, dev_num):

    dev_ide = {}
    for j in roi_center_l:
            dev_ide[j] = 0
            
    num_bit = math.ceil(math.log2(dev_num))
    
    for i in range(num_bit):
        # call device num N to show  ( N >> (num_bit - i) ) ^ 1 to get img
        ori_test_image = cv2.imread(test_seq[i])
        ori_test_image = cv2.GaussianBlur(ori_test_image, (9, 9), 0)
        ori_test = ori_grid(ori_test_image, roi_center_l, G_bound, NW)
        
        for j in roi_center_l:
            dev_ide[j] = dev_ide[j] << 1
            dev_ide[j] += ori_test[j]

    return dev_ide

In [36]:
dev_id = localization(roi, DEV_NUM)
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
fontColor = (255,255,255)
lineType = 2

final = test_image

for i in roi:
    bottomLeftCornerOfText = (math.floor(i.pt[0]),math.floor(i.pt[1]))
    cv2.putText(final, str(dev_id[i]) , bottomLeftCornerOfText, font, fontScale,fontColor,lineType)
    
cv2.imwrite('/Users/lostll/Desktop/180FinalProject/180test/localization.jpg', final) 

True